# Getting started
Based on Rakin's preprocessing to do the item-based collaborative filtering.

## Read & Clean the data 
Here the Data will be stored in Pandas checked for outliers, strange values etc before starting the preprocessing

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import random
import math

# Convert the Data CSV files to pandas
print("Reading Books Data...")
books_data = pd.read_csv("Data/BX-Books.csv", sep=';', on_bad_lines='skip', encoding="latin")
print("Readting Users Data...")
users_data = pd.read_csv("Data/BX-Users.csv", sep=';', on_bad_lines='skip', encoding="latin")
print("Reading Ratings Data...")
book_ratings = pd.read_csv("Data/BX-Book-Ratings.csv", sep=';', on_bad_lines='skip', encoding="latin")
print("Done!")

## Cleaning Books Data
Doing a massive cleaning of books_data.csv. 
1. Removing all the books that have no ISBN
2. Removing all the books that have no author
3. Removing all the books that have no title
4. Removing all the books that have no publisher
5. Removing all the books that have no year of publication or 0 as year of publication
6. Removing all the books that have no image url
7. Removing all the books that have no image url small
8. Removing all the books that have no image url medium
9. Removing all the books that have no image url large
10. Removing all books that have no description

In [ ]:
# Find out which columns have mixed types
print("Total number of rows in the dataset is: ", len(books_data))
print("Total number of unique ISBNs is: ", len(books_data['ISBN'].unique()))

# In Book-Author, there are some values that are not strings. Drop their rows from the dataset
books_data = books_data.loc[books_data['Book-Author'].apply(lambda x: isinstance(x, str)), :]
books_data = books_data.loc[books_data['Year-Of-Publication'].apply(lambda x: isinstance(x, int)), :]
books_data = books_data.loc[books_data['Year-Of-Publication'].apply(lambda x: x != 0), :]
books_data = books_data.loc[books_data['Publisher'].apply(lambda x: isinstance(x, str)), :]
books_data = books_data.loc[books_data['Image-URL-S'].apply(lambda x: isinstance(x, str)), :]
books_data = books_data.loc[books_data['Image-URL-M'].apply(lambda x: isinstance(x, str)), :]
books_data = books_data.loc[books_data['Image-URL-L'].apply(lambda x: isinstance(x, str)), :]
books_data.dropna(inplace=True)
books_data.reset_index(drop=True, inplace=True)
print("Total number of rows in the dataset is: ", len(books_data))

# Grab all the ISBNs that been rated over 20 times in books_data
print("Delete all the ISBNs that have been rated less than 20 times from the dataset...")
ISBNs = book_ratings['ISBN'].value_counts()
ISBNs = ISBNs[ISBNs >= 20]
ISBNs = ISBNs.index.tolist()
books_data = books_data.loc[books_data['ISBN'].apply(lambda x: x in ISBNs), :]
books_data.reset_index(drop=True, inplace=True)
print("Books that have been rated more than 20 times: ", len(books_data))


## Cleaning Book rating and User data

In [ ]:
# Remove all ISBNs that were removed from books_data to book_ratings
print("Removing all the ISBNs that have been removed from the books_data dataset to the book_ratings dataset...")
valid_ISBNs = set(ISBNs)
book_ratings = book_ratings[book_ratings['ISBN'].isin(valid_ISBNs)].reset_index(drop=True)

# Remove the row from book_rating if a user rated a book a 0
print("Removing all the rows from book_ratings if a user rated a book a 0...")
nonzero_mask = book_ratings['Book-Rating'] != 0
book_ratings = book_ratings[nonzero_mask].reset_index(drop=True)

print("Removing all the users that have rated less than 5 books from the book_data dataset and their ISBN...")
users = book_ratings['User-ID'].value_counts()
users = users[users >= 4]
users = users.index.tolist()
book_ratings = book_ratings.loc[book_ratings['User-ID'].apply(lambda x: x in users), :]
book_ratings.reset_index(drop=True, inplace=True)
print("Total number of users that gave a rating of at leat 4 books is: ", len(users))


# If ISBN is in book_ratings and in books_data, keep it in books_data. Otherwise, drop it from books_data
ISBNs = None
ISBNs = book_ratings['ISBN'].value_counts()
ISBNs = ISBNs.index.tolist()
books_data = books_data.loc[books_data['ISBN'].apply(lambda x: x in ISBNs), :]
books_data.reset_index(drop=True, inplace=True)


In [ ]:
# Print out total books, users and ratings
print("Total books: ", len(books_data['ISBN'].unique()))
print("Total users: ", len(book_ratings['User-ID'].unique()))
print("Total ratings: ", len(book_ratings))

## Building the matrix for the recommendation system

In [ ]:
# grab all the ISBNs in the books_data and user s in the book_ratings
ISBNs_filtered = books_data['ISBN'].unique().tolist()
users_filtered = book_ratings['User-ID'].unique().tolist()


# Create a dataframe where the Index is the User-ID and the columns are the ISBNs
print("Creating a dataframe where the Index is the User-ID and the columns are the ISBNs...")
matrix = pd.DataFrame(index=users_filtered, columns=ISBNs_filtered)
matrix.fillna(0, inplace=True)
print("The rows of the matrix are the users: ", matrix.shape[0])


# Fill the matrix with the ratings
for i in range(len(book_ratings)):
  user = book_ratings.iloc[i]['User-ID']
  ISBN = book_ratings.iloc[i]['ISBN']
  rating = book_ratings.iloc[i]['Book-Rating']
  matrix.at[user, ISBN] = rating

# Turn the NaN values to 0
matrix.fillna(0, inplace=True)


In [ ]:
matrix

# item-based collaborative filtering

First do item-based collaborative filtering using traditional method based on users' ratings and cosine similarity.



In [ ]:
# Do cosine similarity bewtween 2 books
def computeItemSimilarity(book1,book2,avg):
   num = 0
   dem1 = 0
   dem2 = 0
   for user in book1.index:
      num += (book1[user] - avg[user]) * (book2[user] - avg[user])
      dem1 += (book1[user] - avg)**2
      dem2 += (book1[user] - avg)**2
   return num / (np.sqrt(dem1) * np.sqrt(dem2))

In [ ]:
item_matrix = pd.DataFrame(index = matrix.columns, columns = matrix.columns)
mean = matrix.mean(axis=1)
for idx in range(len(item_matrix-1)):
    print(idx)
    for idx2 in range(len(item_matrix-1)):
        item_matrix.iloc[idx,idx2] = computeItemSimilarity(matrix.iloc[:,idx],matrix.iloc[:,idx2],mean)
item_matrix